In [2]:
import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

from utils import *

### Fountains

In [3]:
fountains = pd.read_csv(os.getenv("fountains"))
fountains.columns = fountains.columns.str.lower()

In [4]:
fountains = gpd.GeoDataFrame(fountains, geometry=gpd.points_from_xy(fountains.longitude, fountains.latitude))

In [5]:
print(fountains.crs)

None


In [6]:
fountains = fountains.set_crs('epsg:4326')

In [7]:
print(fountains.crs)

epsg:4326


In [8]:
columns = ["precision_localisation", "date_installation", "etat", "x", "y"]
fountains = fountains.drop(columns=columns)

In [9]:
fountains.head(1)

,id,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque,longitude,latitude,geometry
0,1,Saint-Léonard,Delorme,Bocce,rue Loranger/rue Péguy,NaN,-73.589462,45.592012,POINT (-73.58946 45.59201)


### Express bike path (REV)

In [13]:
# rev = gpd.read_file(os.getenv("rev"))
rev = gpd.read_file("rev.geojson")
rev.columns = rev.columns.str.lower()

In [14]:
print(rev.crs)

EPSG:4326


In [16]:
columns = ["id", "nom_arr_vi", "geometry"]
rev = rev.filter(columns)

### calcul the colsest distance between fountain and REV

In [17]:
fountains_utm = fountains.to_crs(epsg=32188)
rev_utm = rev.to_crs(epsg=32188)

In [18]:
print(fountains_utm.crs)
print(rev_utm.crs)

EPSG:32188
EPSG:32188


In [19]:
fountain_w_rev_data = fountains_utm.sjoin_nearest(rev_utm, distance_col="nearest_rev")

In [22]:
print(f"min: {min(fountain_w_rev_data['nearest_rev'])}")
print(f"max: {max(fountain_w_rev_data['nearest_rev'])}")

min: 10.765171370946362
max: 23190.890597142912


In [23]:
m = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [26]:
part = fountain_w_rev_data[fountain_w_rev_data.nom_parc_lieu.str.contains("Royal", na=False)]
for _, row in part.iterrows():
    lat = row.latitude
    lon = row.longitude
    folium.Marker(location=[lat, lon], popup=row["id_left"]).add_to(m)

In [27]:
m

In [28]:
part = rev.copy()
folium.Choropleth(part.geometry, line_weight=2, line_color="blue").add_to(m)

In [29]:
m

In [30]:
fountain_w_rev_data[fountain_w_rev_data.nom_parc_lieu.str.contains("Royal", na=False)]

,id_left,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque,longitude,latitude,geometry,index_right,id_right,nom_arr_vi,nearest_rev
835,849,Ville-Marie,du Mont-Royal,sentier,près du lac des castors,Grands Parcs,-73.599359,45.498969,POINT (297034.910 5039898.870),10,3451,Le Plateau-Mont-Royal,3030.221873
842,856,Ville-Marie,du Mont-Royal,NaN,ave du Parc/ave Duluth Ouest,Grands Parcs,-73.585721,45.514257,POINT (298102.570 5041596.630),10,3451,Le Plateau-Mont-Royal,1024.798587
620,631,Ville-Marie,près du parc du Mont-Royal,NaN,ave des Pins Ouest/rue Peel,au pied de l'escalier,-73.582003,45.504592,POINT (298391.930 5040522.260),12,3453,Le Plateau-Mont-Royal,1786.758000
827,841,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.587419,45.503420,POINT (297968.620 5040392.460),12,3453,Le Plateau-Mont-Royal,2105.848497
828,842,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.586842,45.503720,POINT (298013.690 5040425.750),12,3453,Le Plateau-Mont-Royal,2053.378887
829,843,Ville-Marie,du Mont-Royal,sentier,NaN,Grands Parcs,-73.587095,45.502097,POINT (297993.790 5040245.380),12,3453,Le Plateau-Mont-Royal,2217.138858
830,844,Ville-Marie,du Mont-Royal,chemin,NaN,Grands Parcs,-73.591708,45.502730,POINT (297633.330 5040316.150),12,3453,Le Plateau-Mont-Royal,2352.578419
831,845,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.587162,45.504390,POINT (297988.780 5040500.190),12,3453,Le Plateau-Mont-Royal,2004.539343
832,846,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.588005,45.504188,POINT (297922.910 5040477.850),12,3453,Le Plateau-Mont-Royal,2059.171998
834,848,Ville-Marie,du Mont-Royal,chemin,NaN,Grands Parcs,-73.589275,45.504000,POINT (297823.630 5040457.050),12,3453,Le Plateau-Mont-Royal,2130.723816
